In [1]:
import pandas as pd
from nltk.corpus import stopwords
import json
import numpy as np
import nltk
# nltk.download()

In [379]:
annotations_data = pd.read_csv(r'/Users/vcroopana/Downloads/summer2020/oscar_ads/ip/ad_annotations_oscars_national_edited_1.csv') 
annotations_data = annotations_data[['Advertiser', 'location', 'Ad', 'Keywords', 'Twitter_handle']]
annotations_data['Keywords'] = annotations_data['Ad'].str.cat(annotations_data['Keywords'], sep=" ")\
                                .str.cat(annotations_data['Twitter_handle'], sep=" ")\
                                .str.cat(annotations_data['Advertiser'], sep=" ")# concat the Ad, twitter handle and keywords fields 

annotations_data = annotations_data.fillna("") #location column has NaN values
annotations_data['advtsr_loc'] = annotations_data['Advertiser'].str.cat(annotations_data['location'], sep="_")

twitter_handles = annotations_data['Twitter_handle'].values.tolist()
annotations_data['names'] = annotations_data['Ad'].str.cat(annotations_data['Twitter_handle'], sep=" ")\
                                .str.cat(annotations_data['Advertiser'], sep=" ")
annotations_data.head(2)


,Advertiser,location,Ad,Keywords,Twitter_handle,advtsr_loc,names
0,Cadillac_Escalade,(00:02:44),Escalade,"Escalade Cadillac, driving, red carpet, escala...",cadillac,Cadillac_Escalade_(00:02:44),Escalade cadillac Cadillac_Escalade
1,Kohl's,,Sale,"Sale kohl's, home sale, kohl's cash, kohls.com...",kohls,Kohl's_,Sale kohls Kohl's


In [150]:
stop = stopwords.words('english')
print(len(stop))
stop.extend(["best", "award", "love", "movie", "picture", "words", "actor", "actress", "red", "carpet", "oscar", "oscars"])
print(len(stop))

179
191


In [4]:
#clean tweets
import re
def removeMentions(text):

    textBeforeMention = text.partition("@")[0]
    textAfterMention = text.partition("@")[2]
    textAfterMention =  re.sub(r':', '', textAfterMention) #cadillac join the 31k
    tHandle = textAfterMention.partition(" ")[0].lower() #cadillac
#     if tHandle not in twitter_handles:
#         text = re.sub(r'@\w+', '', text) 
#     else:
#         text = textBeforeMention+ " " + textAfterMention     
    text = textBeforeMention+ " " + textAfterMention  
    return text
    
#TODO issue should've getting converted to shouldve couldn' t -> couldn
def cleanTweet(strinp):
    strinp = re.sub(r'RT', "", strinp) # Remove RT
    strinp = strinp.lower()
    
    stop_removed_list = [word for word in strinp.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_list])        
    text = re.sub('https?://[A-Za-z0-9./]+', ' ', stop_removed) # Remove URLs
    text = removeMentions(text)
    text = re.sub('[^\x00-\x7F]+', ' ', text) # Remove non-ASCII chars.
    text = re.sub('[^a-zA-Z]', ' ', text) # remove all other than alphabet chars 
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # remove all single characters     
    stop_removed_l = [word for word in text.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_l]) 
    return stop_removed

print(cleanTweet("RT @cadillacabc: Joinrt the 31K") )

cadillacabc joinrt k


In [5]:
######## GuidedLDA #############

https://tedunderwood.com/2012/04/07/topic-modeling-made-just-simple-enough/
<br /> topic modelling Vs classificaton: https://monkeylearn.com/topic-analysis/#papers
<br /> https://www.tdktech.com/tech-talks/topic-modeling-explained-lda-to-bayesian-inference
<br />shorttext - https://shorttext.readthedocs.io/en/latest/tutorial_topic.html
<br />naive Bayes - https://monkeylearn.com/topic-analysis/
<br />guidedlda - https://guidedlda.readthedocs.io/en/latest/
<br />guided LDA installation: https://github.com/vi3k6i5/GuidedLDA/issues/26
<br /> guide LDA explanation: https://www.freecodecamp.org/news/how-we-changed-unsupervised-lda-to-semi-supervised-guidedlda-e36a95f3a164/

In [6]:
# Normal LDA without seeding
# model = guidedlda.GuidedLDA(n_topics=14, n_iter=100, random_state=7, refresh=20)
# model.fit(X)


In [137]:
seed_topic_list = [
    ['google', 'loretta'],
    ['verizon'],
    ['apple', 'iphone'],
    ['postmates'],
    ['turbotax', 'tax', 'intuit'],
    ['disney', 'onward', 'mulan', 'memories'],
    ['natalie', 'portman', 'dior'],
    ['cadillac', 'escalade', 'regina', 'king'],
    ['national', 'geographic'],
    ['fires', 'hulu', 'little', 'everywhere', 'witherspoon','fx'],
    ['sale',  'kohl', 'kohls'],
    ['peroni',  'beer', 'birra', 'italia'],
    ['dry', 'canadadry', 'canada'],
    ['galaxy', 'fold', 'samsung', 'phone', 'flip', 'samsungmobileus']
]

In [51]:
seed_topic_list = [
    ['cadillac', 'king', 'driving', 'regina', 'makeyourway', 'make', 'escalade', 'way'],
['sale', 'home', 'kohl', 'cash', 'com', 'kohls'],
['irrisiste', 'bowls', 'applebees', 'good', 'eatin', 'applebee'],
['fixar', 'disney', 'onward', 'march'],
['dry', 'canadadry', 'canada', 'ale', 'real', 'task', 'ginger', 'relax', 'relaxation'],
['necklace', 'macy', 'give', 'macys', 'gifts'],
['marvel', 'studio', 'blackwidow', 'black', 'marvelstudios', 'widow'],
['peroni', 'beautifully', 'peroniusa', 'motorcycle', 'beer', 'birra', 'italia'],
['sale', 'kohl', 'cash', 'com', 'kohls', 'valentine', 'jewerly'],
['network', 'verizon', 'reliable', 'dependable', 'america', 'proven', 'consistnet'],
[ 'gameshow', 'johnsonville', 'sausage', 'way', 'make', 'smoked', 'retro'],
['success', 'cadillac', 'regina', 'king', 'make', 'escalade', 'makeyourway', 'way'],
['martin', 'scorsese', 'kathryn', 'alejandro', 'james', 'bigelow', 'directors', 'perpetual', 'rolex', 'cameron', 'org', 'irritu'],
['say', 'host', 'congrats', 'anything', 'messages', 'mmschocolate', 'help', 'let'],
['galaxy', 'fold', 'samsung', 'foldable', 'phone', 'flip', 'samsungmobileus', 'future', 'change', 'comic', 'strip', 'shape'],
['slow', 'burgers', 'fast', 'postmates', 'postmate', 'subway', 'make', 'floating', 'think'],
['room', 'verizon', 'video', 'er', 'emergency', 'chat', 'unlimited', 'family'],
['us', 'monster', 'lion', 'frozen', 'movies', 'memories', 'bring', 'inc', 'king', 'together', 'disney'],
['tax', 'people', 'intuit', 'turbotax'],
['mmschocolate', 'animated', 'characters', 'blue'],
['fires', 'hulu', 'washington', 'little', 'reese', 'everywhere', 'witherspoon', 'kerry'],
['companion', 'cool', 'american', 'certificate', 'delta', 'express', 'americanexpress', 'grafiti', 'status', 'amex'],
['com', 'farm', 'chsinc', 'chs', 'cooperativeownership'],
['homosexual', 'network', 'verizon', 'people', 'chat', 'group', 'homosexuality', 'rely', 'family'],
['cheese', 'melt', 'beef', 'perfecter', 'made', 'mcdonald', 'pounder', 'fresh', 'perfect', 'quarter', 'mcdonalds', 'sesame'],
['ct', 'nobody', 'cadillac', 'breaking', 'barriers', 'speak', 'make', 'makeyourway', 'glass', 'escalade', 'way'],
['quibi', 'stores', 'president', 'quick', 'big', 'com', 'less', 'minutes', 'bites'],
[ 'chandelier', 'miss', 'rose', 'dior', 'roses'],
['apple', 'verizon', 'iphone', 'mode'],
['park', 'maps', 'jurassic', 'story', 'goonies', 'christmas', 'house', 'google', 'beach'],
['cadillac', 'king', 'driving', 'regina', 'makeyourway', 'make', 'escalade', 'way'],
['mama', 'congrats', 'support', 'emotional', 'proud', 'messages', 'mmschocolate'],
['indeed', 'walked', 'moon', 'spaceship', 'great', 'astraunaut', 'men', 'moments'],
['fires', 'hulu', 'washington', 'littlefireshulu', 'little', 'reese', 'everywhere', 'witherspoon', 'kerry'],
['women', 'hulu', 'rights', 'speech', 'fxonhulu', 'mrsam', 'america', 'mrs', 'fx'],
['cheese', 'melt', 'hulu', 'beef', 'perfecter', 'made', 'mcdonald', 'pounder', 'fresh', 'perfect', 'quarter', 'mcdonalds', 'sesame'],
['discover', 'credit', 'fees', 'annual', 'card', 'fee'],
['railroad', 'quibi', 'com', 'ten', 'episode', 'dessert', 'minutes', 'train'],
['medicine', 'hiv', 'biktarvy', 'living', 'treatment'],
['discover', 'credit', 'accepted', 'card'],
['mulan', 'china', 'liu', 'yifei', 'chinese', 'disney', 'warrier'],
['bar', 'kinderus', 'chocolate', 'bueno', 'kinder'],
['harris', 'maurice', 'flowers', 'surface', 'microsoft'],
['quibi', 'mobile', 'female', 'com', 'phone', 'astronaut'],
['xfi', 'security', 'awesome', 'easy', 'simple', 'xfinity'],
['martin', 'scorsese', 'kathryn', 'alejandro', 'james', 'bigelow', 'directors', 'perpetual', 'rolex', 'cameron', 'org', 'irritu'],
[ 'follow', 'messages', 'favorite', 'mmschocolate'],
['eno', 'wallet', 'capital', 'credit', 'one', 'capitalone', 'highway', 'card'],
['times', 'nyt', 'nytimes', 'slavery', 'worth', 'beach', 'york', 'truth', 'new'],
['finishline', 'cadilac', 'ct', 'nobody', 'running', 'cadillac', 'woman', 'speak'],
['apple', 'verizon', 'iphone', 'mode'],
['quibi', 'mobile', 'leave', 'com', 'phone', 'minutes', 'zombie'],
['groups', 'facebook', 'rock', 'group', 'want', 'together'],
['cynthia', 'natgeo', 'erivo', 'national', 'geographic', 'genius', 'aretha'],
['xcel', 'us', 'energy', 'home', 'xcelenergy', 'smart', 'free', 'together', 'carbon'],
['ct', 'nobody', 'cadillac', 'breaking', 'barriers', 'speak', 'make', 'makeyourway', 'glass', 'escalade', 'way'],
['years', 'optic', 'party', 'colgate', 'dance', 'white', 'toothpaste'],
['norweigian', 'line', 'ncl', 'free', 'cruisenorwegian', 'cruise', 'feel'],
['kellog', 'delight', 'specialk', 'oatmeal', 'special', 'strawberry', 'vanila', 'almond', 'fake', 'chocolatey', 'berries', 'delicious'],
['verizon', 'video', 'connection', 'men', 'angola', 'chat', 'family'],
['selfie', 'say', 'thanks', 'messages', 'mmschocolate', 'let'],
['fx', 'hulu'],
['abc', 'american', 'idol', 'singin', 'season', 'gma', 'americanidol', 'new', 'bus'],
['burger', 'fries', 'eatatperkins', 'pie', 'perkins', 'meal', 'dollars', 'bakery', 'restaurant'],
['cancer', 'verizon', 'dad', 'fight'],
['loretta', 'google', 'old'],
['invite', 'work', 'messages', 'pretending', 'mmschocolate'],
['galaxy', 'fold', 'samsung', 'foldable', 'phone', 'flip', 'samsungmobileus', 'future', 'change', 'comic', 'strip', 'shape'],
['quibi', 'sand', 'quick', 'explorer', 'com', 'minutes', 'swamp'],
['corona', 'coronaextrausa', 'enjoy', 'view', 'premier', 'lower', 'calories', 'carb'],
['reliable', 'verizon', 'consistnet'],  
['martin', 'scorsese', 'kathryn', 'alejandro', 'james', 'bigelow', 'directors', 'perpetual', 'rolex', 'cameron', 'org', 'irritu'],
['flower', 'polar', 'adobe', 'creative', 'paradise', 'suite', 'creativity', 'pride', 'bear', 'astronaut']
    
]

In [ ]:
seed_topic_list = [
    ['commercial', 'cadillac', 'king', 'driving', 'regina', 'makeyourway', 'make', 'escalade', 'way'],
['commercial', 'sale', 'home', 'kohl', 'cash', 'com', 'kohls'],
['commercial', 'irrisiste', 'bowls', 'applebees', 'good', 'eatin', 'applebee'],
['commercial', 'fixar', 'disney', 'onward', 'march'],
['commercial', 'dry', 'canadadry', 'canada', 'ale', 'real', 'task', 'ginger', 'relax', 'relaxation'],
['commercial', 'necklace', 'macy', 'give', 'macys', 'gifts'],
['commercial', 'marvel', 'studio', 'blackwidow', 'black', 'marvelstudios', 'widow'],
['commercial', 'peroni', 'beautifully', 'peroniusa', 'motorcycle', 'beer', 'birra', 'italia'],
['commercial', 'sale', 'kohl', 'cash', 'com', 'kohls', 'valentine', 'jewerly'],
['commercial', 'network', 'verizon', 'reliable', 'dependable', 'america', 'proven', 'consistnet'],
['commercial',  'gameshow', 'johnsonville', 'sausage', 'way', 'make', 'smoked', 'retro'],
['commercial', 'success', 'cadillac', 'regina', 'king', 'make', 'escalade', 'makeyourway', 'way'],
['commercial', 'martin', 'scorsese', 'kathryn', 'alejandro', 'james', 'bigelow', 'directors', 'perpetual', 'rolex', 'cameron', 'org', 'irritu'],
['commercial', 'say', 'host', 'congrats', 'anything', 'messages', 'mmschocolate', 'help', 'let'],
['commercial', 'galaxy', 'fold', 'samsung', 'foldable', 'phone', 'flip', 'samsungmobileus', 'future', 'change', 'comic', 'strip', 'shape'],
['commercial', 'slow', 'burgers', 'fast', 'postmates', 'postmate', 'subway', 'make', 'floating', 'think'],
['commercial', 'room', 'verizon', 'video', 'er', 'emergency', 'chat', 'unlimited', 'family'],
['commercial', 'us', 'monster', 'lion', 'frozen', 'movies', 'memories', 'bring', 'inc', 'king', 'together', 'disney'],
['commercial', 'tax', 'people', 'intuit', 'turbotax'],
['commercial', 'mmschocolate', 'animated', 'characters', 'blue'],
['commercial', 'fires', 'hulu', 'washington', 'little', 'reese', 'everywhere', 'witherspoon', 'kerry'],
['commercial', 'companion', 'cool', 'american', 'certificate', 'delta', 'express', 'americanexpress', 'grafiti', 'status', 'amex'],
# ['commercial', 'com', 'farm', 'chsinc', 'chs', 'cooperativeownership'],
['commercial', 'homosexual', 'network', 'verizon', 'people', 'chat', 'group', 'homosexuality', 'rely', 'family'],
['commercial', 'cheese', 'melt', 'beef', 'perfecter', 'made', 'mcdonald', 'pounder', 'fresh', 'perfect', 'quarter', 'mcdonalds', 'sesame'],
['commercial', 'ct', 'nobody', 'cadillac', 'breaking', 'barriers', 'speak', 'make', 'makeyourway', 'glass', 'escalade', 'way'],
['commercial', 'quibi', 'stores', 'president', 'quick', 'big', 'com', 'less', 'minutes', 'bites'],
['commercial',  'chandelier', 'miss', 'rose', 'dior', 'roses'],
['commercial', 'apple', 'verizon', 'iphone', 'mode'],
# ['commercial', 'park', 'maps', 'jurassic', 'story', 'goonies', 'christmas', 'house', 'google', 'beach'],
['commercial', 'cadillac', 'king', 'driving', 'regina', 'makeyourway', 'make', 'escalade', 'way'],
# ['commercial', 'mama', 'congrats', 'support', 'emotional', 'proud', 'messages', 'mmschocolate'],
['commercial', 'indeed', 'walked', 'moon', 'spaceship', 'great', 'astraunaut', 'men', 'moments'],
['commercial', 'fires', 'hulu', 'washington', 'littlefireshulu', 'little', 'reese', 'everywhere', 'witherspoon', 'kerry'],
# ['commercial', 'women', 'hulu', 'rights', 'speech', 'fxonhulu', 'mrsam', 'america', 'mrs', 'fx'],
['commercial', 'cheese', 'melt', 'hulu', 'beef', 'perfecter', 'made', 'mcdonald', 'pounder', 'fresh', 'perfect', 'quarter', 'mcdonalds', 'sesame'],
['commercial', 'discover', 'credit', 'fees', 'annual', 'card', 'fee'],
['commercial', 'railroad', 'quibi', 'com', 'ten', 'episode', 'dessert', 'minutes', 'train'],
['commercial', 'medicine', 'hiv', 'biktarvy', 'living', 'treatment'],
['commercial', 'discover', 'credit', 'accepted', 'card'],
['commercial', 'mulan', 'china', 'liu', 'yifei', 'chinese', 'disney', 'warrier'],
['commercial', 'bar', 'kinderus', 'chocolate', 'bueno', 'kinder'],
['commercial', 'harris', 'maurice', 'flowers', 'surface', 'microsoft'],
['commercial', 'quibi', 'mobile', 'female', 'com', 'phone', 'astronaut'],
['commercial', 'xfi', 'security', 'awesome', 'easy', 'simple', 'xfinity'],
# ['commercial', 'martin', 'scorsese', 'kathryn', 'alejandro', 'james', 'bigelow', 'directors', 'perpetual', 'rolex', 'cameron', 'org', 'irritu'],
['commercial',  'follow', 'messages', 'favorite', 'mmschocolate'],
['commercial', 'eno', 'wallet', 'capital', 'credit', 'one', 'capitalone', 'highway', 'card'],
['commercial', 'times', 'nyt', 'nytimes', 'slavery', 'worth', 'beach', 'york', 'truth', 'new'],
# ['commercial','finishline', 'cadilac', 'ct', 'nobody', 'running', 'cadillac', 'woman', 'speak'],
['commercial', 'apple', 'verizon', 'iphone', 'mode'],
['commercial', 'quibi', 'mobile', 'leave', 'com', 'phone', 'minutes', 'zombie'],
['commercial', 'groups', 'facebook', 'rock', 'group', 'want', 'together'],
['commercial', 'cynthia', 'natgeo', 'erivo', 'national', 'geographic', 'genius', 'aretha'],
['commercial', 'xcel', 'us', 'energy', 'home', 'xcelenergy', 'smart', 'free', 'together', 'carbon'],
['commercial', 'ct', 'nobody', 'cadillac', 'breaking', 'barriers', 'speak', 'make', 'makeyourway', 'glass', 'escalade', 'way'],
['commercial', 'years', 'optic', 'party', 'colgate', 'dance', 'white', 'toothpaste'],
['commercial', 'norweigian', 'line', 'ncl', 'free', 'cruisenorwegian', 'cruise', 'feel'],
['commercial', 'kellog', 'delight', 'specialk', 'oatmeal', 'special', 'strawberry', 'vanila', 'almond', 'fake', 'chocolatey', 'berries', 'delicious'],
['commercial', 'verizon', 'video', 'connection', 'men', 'angola', 'chat', 'family'],
['commercial', 'selfie', 'say', 'thanks', 'messages', 'mmschocolate', 'let'],
['commercial', 'fx', 'hulu'],
['commercial', 'abc', 'american', 'idol', 'singin', 'season', 'gma', 'americanidol', 'new', 'bus'],
['commercial', 'burger', 'fries', 'eatatperkins', 'pie', 'perkins', 'meal', 'dollars', 'bakery', 'restaurant'],
['commercial', 'cancer', 'verizon', 'dad', 'fight'],
['commercial', 'loretta', 'google', 'old'],
['commercial', 'invite', 'work', 'messages', 'pretending', 'mmschocolate'],
['commercial', 'galaxy', 'fold', 'samsung', 'foldable', 'phone', 'flip', 'samsungmobileus', 'future', 'change', 'comic', 'strip', 'shape'],
['commercial', 'quibi', 'sand', 'quick', 'explorer', 'com', 'minutes', 'swamp'],
['commercial', 'corona', 'coronaextrausa', 'enjoy', 'view', 'premier', 'lower', 'calories', 'carb'],
# ['commercial', 'reliable', 'verizon', 'consistnet'],  
['commercial', 'martin', 'scorsese', 'kathryn', 'alejandro', 'james', 'bigelow', 'directors', 'perpetual', 'rolex', 'cameron', 'org', 'irritu'],
['commercial', 'flower', 'polar', 'adobe', 'creative', 'paradise', 'suite', 'creativity', 'pride', 'bear', 'astronaut']
    
]

In [163]:
seed_topic_list = [
    ['commercial', 'cadillac', 'escalade'],
#     ['commercial', 'success', 'cadillac', 'regina', 'king', 'make', 'escalade', 'makeyourway', 'way'],
# ['commercial', 'ct', 'nobody', 'cadillac', 'breaking', 'barriers', 'speak', 'make', 'makeyourway', 'glass', 'escalade', 'way'],
# ['commercial', 'cadillac', 'king', 'driving', 'regina', 'makeyourway', 'make', 'escalade', 'way'],
# ['commercial', 'ct', 'nobody', 'cadillac', 'breaking', 'barriers', 'speak', 'make', 'makeyourway', 'glass', 'escalade', 'way'],

['commercial', 'sale', 'kohl','kohls'],
# ['commercial', 'sale', 'kohl', 'cash', 'com', 'kohls', 'valentine', 'jewerly'],

['commercial',  'applebees',  'applebee'],
['commercial', 'fixar', 'disney', 'onward'],
['commercial', 'dry', 'canadadry', 'canada', 'ale', 'ginger', 'relax', 'relaxation'],
['commercial', 'necklace', 'macy','macys', 'gifts'],
['commercial', 'marvel', 'studio', 'blackwidow', 'black', 'marvelstudios', 'widow'],
['commercial', 'peroni','peroniusa', 'motorcycle', 'beer', 'birra', 'italia'],
['commercial', 'network', 'verizon', 'reliable', 'dependable', 'america', 'proven', 'consistent'],
# ['commercial',  'gameshow', 'johnsonville', 'sausage', 'way', 'make', 'smoked', 'retro'],
# ['commercial', 'martin', 'scorsese', 'kathryn', 'alejandro', 'james', 'bigelow', 'directors', 'perpetual', 'rolex', 'cameron', 'org', 'irritu'],
['commercial',  'mmschocolate'],
# ['commercial', 'mmschocolate', 'animated', 'characters', 'blue'],
#         ['commercial',  'follow', 'messages', 'favorite', 'mmschocolate'],
# ['commercial', 'selfie', 'say', 'thanks', 'messages', 'mmschocolate', 'let'],
#['commercial', 'invite', 'work', 'messages', 'pretending', 'mmschocolate'],

['commercial', 'galaxy', 'fold', 'samsung', 'foldable', 'phone', 'flip', 'samsungmobileus'],
# ['commercial', 'galaxy', 'fold', 'samsung', 'foldable', 'phone', 'flip', 'samsungmobileus', 'future', 'change', 'comic', 'strip', 'shape'],

['commercial', 'burgers', 'postmates', 'postmate', 'subway'],
['commercial',  'verizon', 'video', 'emergency', 'chat', 'unlimited', 'family'],
# ['commercial', 'homosexual', 'network', 'verizon', 'people', 'chat', 'group', 'homosexuality', 'rely', 'family'],
# ['commercial', 'verizon', 'video', 'connection', 'men', 'angola', 'chat', 'family'],
# ['commercial', 'cancer', 'verizon', 'dad', 'fight'],
# ['commercial', 'reliable', 'verizon', 'consistnet'],  

['commercial', 'apple', 'iphone', 'mode'],
# ['commercial', 'apple', 'verizon', 'iphone', 'mode'],

['commercial', 'lion', 'frozen', 'memories',  'disney'],
['commercial', 'tax', 'people', 'intuit', 'turbotax'],

['commercial', 'fires', 'hulu', 'little', 'reese', 'everywhere'],
# ['commercial', 'fires', 'hulu', 'washington', 'littlefireshulu', 'little', 'reese', 'everywhere', 'witherspoon', 'kerry'],
['commercial', 'fx', 'hulu'],

['commercial', 'companion', 'cool', 'american', 'delta', 'express', 'americanexpress', 'amex'],
# ['commercial', 'com', 'farm', 'chsinc', 'chs', 'cooperativeownership'],
['commercial', 'cheese', 'melt', 'beef', 'mcdonald', 'pounder', 'fresh', 'perfect', 'quarter', 'mcdonalds', 'sesame'],
# ['commercial', 'cheese', 'melt', 'hulu', 'beef', 'perfecter', 'made', 'mcdonald', 'pounder', 'fresh', 'perfect', 'quarter', 'mcdonalds', 'sesame'],

['commercial', 'quibi', 'stores', 'president', 'bites'],
['commercial', 'railroad', 'quibi', 'com', 'ten', 'episode', 'dessert', 'train'],
# ['commercial', 'quibi', 'mobile', 'female', 'com', 'phone', 'astronaut'],
# ['commercial', 'quibi', 'mobile', 'leave', 'com', 'phone', 'minutes', 'zombie'],
# ['commercial', 'quibi', 'sand', 'quick', 'explorer', 'com', 'minutes', 'swamp'],

['commercial',  'chandelier', 'miss', 'rose', 'dior', 'roses'],
    
# ['commercial', 'park', 'maps', 'jurassic', 'story', 'goonies', 'christmas', 'house', 'google', 'beach'],
# ['commercial', 'mama', 'congrats', 'support', 'emotional', 'proud', 'messages', 'mmschocolate'],
['commercial', 'indeed', 'moon', 'spaceship', 'astraunaut'],
# ['commercial', 'women', 'hulu', 'rights', 'speech', 'fxonhulu', 'mrsam', 'america', 'mrs', 'fx'],

['commercial', 'discover', 'credit', 'fees', 'card', 'fee'],
#     ['commercial', 'discover', 'credit', 'accepted', 'card'],
['commercial', 'medicine', 'hiv', 'biktarvy', 'treatment'],

['commercial', 'mulan', 'china', 'liu', 'yifei', 'chinese', 'disney', 'warrier'],
['commercial', 'bar', 'kinderus', 'chocolate', 'bueno', 'kinder'],
['commercial', 'harris', 'maurice', 'flowers', 'surface', 'microsoft'],
['commercial', 'xfi', 'security', 'awesome', 'easy', 'simple', 'xfinity'],
# ['commercial', 'martin', 'scorsese', 'kathryn', 'alejandro', 'james', 'bigelow', 'directors', 'perpetual', 'rolex', 'cameron', 'org', 'irritu'],

['commercial', 'eno', 'wallet', 'capital', 'capitalone', 'highway'],
['commercial', 'times', 'nyt', 'nytimes', 'slavery', 'worth', 'beach', 'york', 'truth', 'new'],
# ['commercial','finishline', 'cadilac', 'ct', 'nobody', 'running', 'cadillac', 'woman', 'speak'],


['commercial', 'groups', 'facebook', 'rock', 'group'],
['commercial', 'cynthia', 'natgeo', 'erivo', 'national', 'geographic'],
['commercial', 'xcel', 'us', 'energy', 'home', 'xcelenergy', 'smart', 'free', 'together', 'carbon'],

['commercial', 'colgate', 'white', 'toothpaste'],
['commercial', 'norweigian',  'cruisenorwegian', 'cruise'],
    
['commercial', 'kellog',  'specialk', 'oatmeal','strawberry', 'vanila', 'almond', 'fake', 'chocolatey', 'berries', 'delicious'],

['commercial', 'abc', 'american', 'idol', 'gma', 'americanidol', 'bus'],
['commercial', 'burger', 'fries', 'eatatperkins', 'pie', 'perkins', 'meal', 'dollars', 'bakery', 'restaurant'],

['commercial', 'loretta', 'google', 'old'],

['commercial', 'corona', 'coronaextrausa', 'enjoy', 'view', 'premier', 'lower', 'calories', 'carb'],


['commercial', 'directors',  'rolex'],
['commercial', 'flower', 'polar', 'adobe', 'creative', 'paradise', 'suite', 'creativity', 'pride', 'bear']
    
]

In [479]:
seed_topic_list = [
    [ 'cadillac', 'escalade'],
#     [ 'success', 'cadillac', 'regina', 'king', 'make', 'escalade', 'makeyourway', 'way'],
# [ 'ct', 'nobody', 'cadillac', 'breaking', 'barriers', 'speak', 'make', 'makeyourway', 'glass', 'escalade', 'way'],
# [ 'cadillac', 'king', 'driving', 'regina', 'makeyourway', 'make', 'escalade', 'way'],
# [ 'ct', 'nobody', 'cadillac', 'breaking', 'barriers', 'speak', 'make', 'makeyourway', 'glass', 'escalade', 'way'],

['sale', 'kohl','kohls'],
# [ 'sale', 'kohl', 'cash', 'com', 'kohls', 'valentine', 'jewerly'],

[  'applebees',  'applebee'],
['fixar', 'disney', 'onward'],
[ 'dry', 'canadadry', 'canada', 'ale', 'ginger', 'relax', 'relaxation'],
[ 'necklace', 'macy','macys', 'gifts'],
[ 'marvel', 'studio', 'blackwidow', 'black', 'marvelstudios', 'widow'],
[ 'peroni','peroniusa', 'motorcycle', 'beer', 'birra', 'italia'],
[ 'network', 'verizon', 'reliable', 'dependable', 'america', 'proven', 'consistent'],
# [  'gameshow', 'johnsonville', 'sausage', 'way', 'make', 'smoked', 'retro'],
# [ 'martin', 'scorsese', 'kathryn', 'alejandro', 'james', 'bigelow', 'directors', 'perpetual', 'rolex', 'cameron', 'org', 'irritu'],
[  'mmschocolate'],
# [ 'mmschocolate', 'animated', 'characters', 'blue'],
#         [  'follow', 'messages', 'favorite', 'mmschocolate'],
# [ 'selfie', 'say', 'thanks', 'messages', 'mmschocolate', 'let'],
#[ 'invite', 'work', 'messages', 'pretending', 'mmschocolate'],

[ 'galaxy', 'fold', 'samsung', 'foldable', 'phone', 'flip', 'samsungmobileus'],
# [ 'galaxy', 'fold', 'samsung', 'foldable', 'phone', 'flip', 'samsungmobileus', 'future', 'change', 'comic', 'strip', 'shape'],

[ 'burgers', 'postmates', 'postmate', 'subway'],
[  'verizon', 'video', 'emergency', 'chat', 'unlimited', 'family'],
# [ 'homosexual', 'network', 'verizon', 'people', 'chat', 'group', 'homosexuality', 'rely', 'family'],
# [ 'verizon', 'video', 'connection', 'men', 'angola', 'chat', 'family'],
# [ 'cancer', 'verizon', 'dad', 'fight'],
# [ 'reliable', 'verizon', 'consistnet'],  

[ 'apple', 'iphone', 'mode'],
# [ 'apple', 'verizon', 'iphone', 'mode'],

[ 'lion', 'frozen', 'memories',  'disney'],
[ 'tax', 'people', 'intuit', 'turbotax'],

[ 'fires', 'hulu', 'little', 'reese', 'everywhere'],
# [ 'fires', 'hulu', 'washington', 'littlefireshulu', 'little', 'reese', 'everywhere', 'witherspoon', 'kerry'],
[ 'fx', 'hulu'],

[ 'companion', 'cool', 'american', 'delta', 'express', 'americanexpress', 'amex'],
# [ 'com', 'farm', 'chsinc', 'chs', 'cooperativeownership'],
[ 'cheese', 'melt', 'beef', 'mcdonald', 'pounder', 'fresh', 'perfect', 'quarter', 'mcdonalds', 'sesame'],
# [ 'cheese', 'melt', 'hulu', 'beef', 'perfecter', 'made', 'mcdonald', 'pounder', 'fresh', 'perfect', 'quarter', 'mcdonalds', 'sesame'],

[ 'quibi', 'stores', 'president', 'bites'],
[ 'railroad', 'quibi', 'com', 'ten', 'episode', 'dessert', 'train'],
# [ 'quibi', 'mobile', 'female', 'com', 'phone', 'astronaut'],
# [ 'quibi', 'mobile', 'leave', 'com', 'phone', 'minutes', 'zombie'],
# [ 'quibi', 'sand', 'quick', 'explorer', 'com', 'minutes', 'swamp'],

[  'chandelier', 'miss', 'rose', 'dior', 'roses'],
    
# [ 'park', 'maps', 'jurassic', 'story', 'goonies', 'christmas', 'house', 'google', 'beach'],
# [ 'mama', 'congrats', 'support', 'emotional', 'proud', 'messages', 'mmschocolate'],
[ 'indeed', 'moon', 'spaceship', 'astraunaut'],
# [ 'women', 'hulu', 'rights', 'speech', 'fxonhulu', 'mrsam', 'america', 'mrs', 'fx'],

[ 'discover', 'credit', 'fees', 'card', 'fee'],
#     [ 'discover', 'credit', 'accepted', 'card'],
[ 'medicine', 'hiv', 'biktarvy', 'treatment'],

[ 'mulan', 'china', 'liu', 'yifei', 'chinese', 'disney', 'warrier'],
[ 'bar', 'kinderus', 'chocolate', 'bueno', 'kinder'],
[ 'harris', 'maurice', 'flowers', 'surface', 'microsoft'],
[ 'xfi', 'security', 'awesome', 'easy', 'simple', 'xfinity'],
# [ 'martin', 'scorsese', 'kathryn', 'alejandro', 'james', 'bigelow', 'directors', 'perpetual', 'rolex', 'cameron', 'org', 'irritu'],

[ 'eno', 'wallet', 'capital', 'capitalone', 'highway'],
[ 'times', 'nyt', 'nytimes', 'slavery', 'worth', 'beach', 'york', 'truth', 'new'],
# ['commercial','finishline', 'cadilac', 'ct', 'nobody', 'running', 'cadillac', 'woman', 'speak'],


[ 'groups', 'facebook', 'rock', 'group'],
[ 'cynthia', 'natgeo', 'erivo', 'national', 'geographic'],
[ 'xcel', 'us', 'energy', 'home', 'xcelenergy', 'smart', 'free', 'together', 'carbon'],

[ 'colgate', 'white', 'toothpaste'],
[ 'norweigian',  'cruisenorwegian', 'cruise'],
    
[ 'kellog',  'specialk', 'oatmeal','strawberry', 'vanila', 'almond', 'fake', 'chocolatey', 'berries', 'delicious'],

[ 'abc', 'american', 'idol', 'gma', 'americanidol', 'bus'],
[ 'burger', 'fries', 'eatatperkins', 'pie', 'perkins', 'meal', 'dollars', 'bakery', 'restaurant'],

[ 'loretta', 'google', 'old'],

[ 'corona', 'coronaextrausa', 'enjoy', 'view', 'premier', 'lower', 'calories', 'carb'],


[ 'directors',  'rolex'],
[ 'flower', 'polar', 'adobe', 'creative', 'paradise', 'suite', 'creativity', 'pride', 'bear']
    
]


In [481]:
import guidedlda as guidedlda
import numpy as np

def buildWordVector(train_data, word2id):
    X = np.zeros((len(train_data), len(word2id)), dtype = 'int64')

    for i in range(0, len(train_data)):
        doc_words = train_data[i].split()
        for word in doc_words:  
            if word in word2id:
                X[i][word2id[word]] += 1
    return X

def buildGuidedLDAModel(train_data, seed_topic_list, n_topics, seed_confidence, 
                        n_iter, n_top_words, random_state=7, refresh=1):
    
    ad_words = []
    for keywrds in train_data:
        keywordsSet = list(set(keywrds.split()))
        ad_words.extend(keywordsSet)

    ad_words = list(set(ad_words))
    print("Length of ad keywords: "+ str(len(ad_words)))

    word2id = dict((v, idx) for idx, v in enumerate(ad_words)) #dict of word: id

    X = buildWordVector(train_data, word2id)     #Build X - input data

    seed_topics = {}     #Build Seed topics list
    for t_id, st in enumerate(seed_topic_list):
        for word in st:
            if word in word2id:
                 seed_topics[word2id[word]] = t_id

    model = guidedlda.GuidedLDA(n_topics, n_iter, random_state= random_state, refresh= refresh)
    model.fit(X, seed_topics=seed_topics, seed_confidence= seed_confidence)

    topic_vector = model.topic_word_ #topic vector [[0.00059559 0..0.00059559]..[0.00263852 ...0.00263852]]
    topic_words = []

    for i, topic_dist in enumerate(topic_vector):
         topic_word = np.array(ad_words)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
         topic_words.append(topic_word.tolist())
         print('Topic {}: {}'.format(i, ' '.join(topic_word)))
    
    return model, topic_words, word2id

In [528]:
#ip required: n_topics, topic_words
def get_max_topic_id(topic_vector, x):
    if x['max_prob']<=1:
        return topic_vector.columns.get_loc(x['max_topic']) 
    else:
        return None
def get_max_topic_words(topic_words, topic_id):
    if topic_id is not None:
        return topic_words[topic_id] 
    else:
        return None

def buildTopicVectors(doc_topic, n_topics, topic_words):
    
    columns_label = ['topic {}'.format(i) for i in range(n_topics)]  # number of topics
    topic_vector = pd.DataFrame(doc_topic, columns = columns_label)  #dataframe of doc-topics

    topic_vector['max_prob'] = topic_vector.max(axis=1)
    topic_vector['max_topic'] = topic_vector.idxmax(axis =1)
#     topic_vector['max_topic_id'] = topic_vector['max_topic'].apply(lambda x: topic_vector.columns.get_loc(x))
    topic_vector['max_topic_id'] = topic_vector.apply(lambda x: get_max_topic_id(topic_vector, x), axis = 1)
#     topic_vector['topic_words'] = topic_vector['max_topic_id'].apply(lambda x: topic_words[x] if x is not None 
#                                                                     else None)
    topic_vector['topic_words'] = topic_vector['max_topic_id'].apply(lambda x: get_max_topic_words(topic_words, x))
    
    return topic_vector

def get_conf_matrix(ad_manual, max_prob, thresh_prob):
    res = ""
    if(ad_manual=='none' and max_prob >= thresh_prob):
        res = 'FP'
    elif(ad_manual=='none' and max_prob < thresh_prob):
        res = 'TN'
    elif(ad_manual!='none' and max_prob >= thresh_prob):
        res = 'TP'
    elif(ad_manual!='none' and max_prob < thresh_prob):
        res = 'FN'
    return res

def computeAccuracy(result):    
    n_tp = result[result['conf_matrix'] == 'TP'].shape[0]
    n_fp = result[result['conf_matrix'] == 'FP'].shape[0]
    n_fn = result[result['conf_matrix'] == 'FN'].shape[0]
    n_tn = result[result['conf_matrix'] == 'TN'].shape[0]
    print("n_tp:"+ str(n_tp)+" n_fp:"+ str(n_fp)+" n_fn:"+ str(n_fn)+" n_tn:"+ str(n_tn))

    precision = n_tp/(n_tp+ n_fp)
    recall = n_tp/(n_tp+ n_fn)
    f_measure = (2*precision*recall)/ (precision+recall)

    return precision, recall, f_measure


def removePOS(taggedText, includePOS):
    op = ' '.join([pair[0] for pair in taggedText if pair[1] in includePOS])
    return op


In [531]:
#training data for the model
import logging

main = logging.getLogger()
main.setLevel(logging.ERROR)

# annotations_data['Keywords'] = annotations_data['Keywords'].apply(lambda x: x+" commercial")
train_data = annotations_data['Keywords'].apply(lambda x: cleanTweet(x))

# annotations_data['names_edited'] = annotations_data['names'].apply(lambda x: x+" commercial")
# train_data = annotations_data['names_edited'].apply(lambda x: cleanTweet(x))

train_data_pos = train_data.apply(lambda x: removePOS(nltk.pos_tag(x.split()), ('NN') ))

seed_topic_pos = []
for seeds in seed_topic_list:
    tagged = nltk.pos_tag(seeds)
    seed_topic_pos.append(removePOS(tagged, ('NN')))
    
# print(seed_topic_pos)   

model, topic_words, word2id = buildGuidedLDAModel(
                                                  train_data, 
#                                                 train_data_pos, 
                                                  seed_topic_list,
#                                                 seed_topic_pos,
                                                  n_topics= 67, 
                                                  seed_confidence = 1, 
                        n_iter= 500, n_top_words = 10, random_state=7, refresh=1)


Length of ad keywords: 377
Topic 0: cadillac escalade makeyourway way make regina king glass driving studio
Topic 1: kohl kohls com sale cash valentine jewerly home astraunaut studio
Topic 2: jurassic samsungmobileus idol studio escalade emotional mobile ten inc quick
Topic 3: angola coronaextrausa americanidol season samsungmobileus studio escalade emotional mobile ten
Topic 4: canada dry ale real escalade emotional mobile ten inc quick
Topic 5: gifts give macy necklace macys jurassic astraunaut escalade emotional mobile
Topic 6: widow black marvel studio blackwidow rely marvelstudios jurassic escalade emotional
Topic 7: peroni italia peroniusa beautifully birra beer sand motorcycle inc one
Topic 8: er unlimited emergency room proud emotional inc one idol studio
Topic 9: chs chsinc cooperativeownership farm escalade emotional mobile ten inc quick
Topic 10: phone foldable samsung fold samsungmobileus galaxy flip comic shape strip
Topic 11: aretha geographic national cynthia natgeo geni

In [265]:

man_ann_data_raw = pd.read_csv(r'/Users/vcroopana/Downloads/summer2020/oscar_ads/op/jaccard/op_jacc_5K_gender_sampled.csv')    
man_ann_data = man_ann_data_raw.dropna(subset= ['ad_manual'])
man_ann_data.head(1)


,index,tweet_id_str,created_at,text,retweeted_status,text_clean,ad_mentioned,ad_manual,ad_keywords,user_id,user_name,gender_firstname,description,gender_text_200_tweets,comm,ad_clean,ad_gender
2499,373,1.226680e+18,Mon Feb 10 01:23:17 +0000 2020,RT @frontlinepbs: The #Oscars start in just a ...,{'created_at': 'Mon Feb 10 00:54:11 +0000 2020...,oscars start minutes sending luck wishes team ...,quibi.com_(1:16:30),none,"['bites', 'stores', 'quibi', 'minutes', 'big',...",20006234,WETA,NaN,"Public television channels TV 26, WETA HD, WET...",female,0,none,NaN


In [532]:
test_data = [cleanTweet(doc) for doc in man_ann_data['text_clean']]
X = buildWordVector(test_data, word2id)


doc_topic = model.transform(X)
# print(doc_topic)
topic_vector = buildTopicVectors(doc_topic, 67, topic_words)
# topic_vector.round(2).head(2)

man_ann_data = man_ann_data.reset_index(drop=True)

result = pd.concat([man_ann_data, topic_vector], axis=1)


In [537]:
result['conf_matrix'] = result.apply(lambda x: get_conf_matrix(x['ad_manual'], x['max_prob'], 
                                                               thresh_prob = 0.84), axis =1)
precision, recall, f_measure = computeAccuracy(result)
print("precision: "+ str(np.round(precision, 3))+" recall: "+ str(np.round(recall,3))+" f_measure: "
      + str(np.round(f_measure,3)))

op_name = '/Users/vcroopana/Downloads/summer2020/oscar_ads/op/topicmodelling/op_mann_lda_67_topics.csv'


n_tp:308 n_fp:189 n_fn:381 n_tn:1623
precision: 0.62 recall: 0.447 f_measure: 0.519


In [518]:
cols_reqd = ['text', 'text_clean', 'ad_manual', 'ad_keywords', 'max_prob', 'max_topic_id', 'topic_words', 'conf_matrix']
result[cols_reqd].to_csv(op_name)

In [519]:
result.head(1)

,index,tweet_id_str,created_at,text,retweeted_status,text_clean,ad_mentioned,ad_manual,ad_keywords,user_id,...,topic 62,topic 63,topic 64,topic 65,topic 66,max_prob,max_topic,max_topic_id,topic_words,conf_matrix
0,373,1.226680e+18,Mon Feb 10 01:23:17 +0000 2020,RT @frontlinepbs: The #Oscars start in just a ...,{'created_at': 'Mon Feb 10 00:54:11 +0000 2020...,oscars start minutes sending luck wishes team ...,quibi.com_(1:16:30),none,"['bites', 'stores', 'quibi', 'minutes', 'big',...",20006234,...,6.946954e-310,6.946954e-310,6.946954e-310,6.946954e-310,6.946954e-310,6.946988e-310,topic 1,1,"[mrs, fxonhulu, speech, work, hulu, mrsam, fol...",TN


In [413]:
#Approaches tried:
#Trained on test data without using ad keywords
# Trained adding commerical to keywwords i.e training data and see topic list 
# trained without commercial in keywords i.e training data and seed topic list : f1 score: 0.53 with 0.84 thresh prob
# n_top_words = 3, fscore = 0.544 at 0.9 thresh prob
# 35 topics, 3 words, 900 iters, 0.91 prob 0.551 f score; precision: 0.601 recall: 0.509 f_measure: 0.551
# 35 topics, 3 words, 200 iters, 0.92 prob 0.562 f score; precision: 0.627 recall: 0.509 f_measure: 0.562


# google (145/209), disney() - too many false positives
# False Negatives ?

# ntopics = 20, 3 words, 200 iters, 0.97 prob, 
# n_tp:354 n_fp:191 n_fn:335
# precision: 0.65 recall: 0.514 f_measure: 0.574

# google (140/191) - too many false positives

#67 topics:
# n_tp:313 n_fp:177 n_fn:376 n_tn:1635
# precision: 0.639 recall: 0.454 f_measure: 0.531


In [520]:
import pyLDAvis.gensim

def displayModel(model, corpus, dictionary):
    
    lda_model_display = pyLDAvis.gensim.prepare(model, corpus, dictionary, sort_topics=True, mds='mmds')
    pyLDAvis.show(lda_model_display)

In [ ]:
displayModel(model, , word2id)